久保拓弥『データ解析のための統計モデリング入門』をJuliaでやってみます。

* 久保さんの書籍ページ：http://hosho.ees.hokudai.ac.jp/~kubo/ce/IwanamiBook.html
* Amazon.co.jp：https://amzn.to/2Cg5vDJ

# 第7章「一般化線形混合モデル（GLMM） ---個体差のモデリング---」（途中）


In [65]:
using DataFrames
using CSV
using Distributions
using GLM
using Plots
pyplot()
using StatsPlots